In [1]:
import celltypist
from celltypist import models
import scanpy as sc
import pandas as pd 
import numpy as np
import anndata
import re
import h5py
import scipy.sparse as scs
import concurrent.futures
import scanpy.external as sce

In [6]:
pip install scrublet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 17.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=79166 sha256=d4663e1ffd035bfb96db91f713e260398c745937269b085b03f93595166fa037
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy
Note: you may need to restart the kernel to use updated packages.


In [2]:
def read_mat(h5_con):
    mat = scs.csc_matrix(
        (h5_con['matrix']['data'][:], # Count values
         h5_con['matrix']['indices'][:], # Row indices
         h5_con['matrix']['indptr'][:]), # Pointers for column positions
        shape = tuple(h5_con['matrix']['shape'][:]) # Matrix dimensions
    )
    return mat


def read_obs(h5con):
    bc = h5con['matrix']['barcodes'][:]
    bc = [x.decode('UTF-8') for x in bc]

    # Initialized the DataFrame with cell barcodes
    obs_df = pd.DataFrame({ 'barcodes' : bc })

    # Get the list of available metadata columns
    obs_columns = h5con['matrix']['observations'].keys()

    # For each column
    for col in obs_columns:
        # Read the values
        values = h5con['matrix']['observations'][col][:]
        # Check for byte storage
        if(isinstance(values[0], (bytes, bytearray))):
            # Decode byte strings
            values = [x.decode('UTF-8') for x in values]
        # Add column to the DataFrame
        obs_df[col] = values
    
    return obs_df
# define a function to construct anndata object from a h5 file
def read_h5_anndata(h5_file):
    h5_con = h5py.File(h5_file, mode = 'r')
    # extract the expression matrix
    mat = read_mat(h5_con)
    # extract gene names
    genes = h5_con['matrix']['features']['name'][:]
    genes = [x.decode('UTF-8') for x in genes]
    # extract metadata
    obs_df = read_obs(h5_con)
    # construct anndata
    adata = anndata.AnnData(mat.T,
                             obs = obs_df)
    # make sure the gene names aligned
    adata.var_names = genes

    adata.var_names_make_unique()
    return adata
def get_last_pattern(inputstr):
    pattern = r"[^/]+(?=$)"
    match = re.search(pattern, inputstr)
    if match:
        return match.group(0)
    else:
        return ""

In [3]:
meta_data=pd.read_csv("/home//jupyter/hise_meta_data_2023-11-02.csv")

In [4]:
def process_file(file_name):
    result = read_h5_anndata(file_name)
    sc.external.pp.scrublet(result)
    return result.obs[['barcodes','predicted_doublet','doublet_score']]

In [7]:
from concurrent.futures import ThreadPoolExecutor

results = []

with ThreadPoolExecutor(max_workers=20) as executor:  
    for result in executor.map(process_file, meta_data['file.path']):
        results.append(result)

Automatically set threshold at doublet score = 0.35
Detected doublet rate = 1.7%
Estimated detectable doublet fraction = 29.1%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 6.0%
Automatically set threshold at doublet score = 0.68
Detected doublet rate = 0.1%
Estimated detectable doublet fraction = 3.7%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 3.6%
Automatically set threshold at doublet score = 0.36
Detected doublet rate = 1.2%
Estimated detectable doublet fraction = 36.3%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 3.3%
Automatically set threshold at doublet score = 0.37
Detected doublet rate = 1.0%
Estimated detectable doublet fraction = 27.7%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 3.7%
Automatically set threshold at doublet score = 0.72
Detected doublet rate = 0.0%
Estimated detectable doublet fraction = 0.3%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 3.2%
Automatically set threshold at doublet score = 0.69
Detected double

In [8]:
final_result = pd.concat(results, ignore_index=True)

In [9]:
final_result.to_parquet('doublet_score.parquet')

In [10]:
final_result['predicted_doublet'].value_counts()

predicted_doublet
False    2066388
True       27399
Name: count, dtype: int64

In [12]:
27399/(2066388+27399)

0.013085858303638336